# Appendix
https://en.wikipedia.org/wiki/2048_(video_game) 
https://www.geeksforgeeks.org/2048-game-in-python/

problem statement : https://drive.google.com/file/d/1-l9t-sKYWD_hrAAmfzJFmv2X151lMsEo/view?usp=drive_link

IMPLEMENTED GAME
4X4 GRID (implemented as a numpy grid)
4 switches to move all up/all down /all right /all left
(moves with the keyboard up/l/d/r buttons)


# game user manual 
i desgned first the basic version of the game with help of the given regulations 
then i added the required moves counter to show the number of moves taken

on playing i realized to add an undo feature to go back and check out other possibiities in each game 
(which works by clicking z) , and to not waste space i have ensured maximum 5 undo's
and also escape button works as to exit the game

![alt text](image.png)

# game

In [10]:
import pygame
import random
from math import log
import numpy as np

In [11]:
# --- Constants ---
GRID_SIZE = 4
TILE_SIZE = 100
MARGIN = 10
WIDTH = (TILE_SIZE + MARGIN) * GRID_SIZE
HEIGHT = (TILE_SIZE + MARGIN) * GRID_SIZE + 50 #Added 50 for moves display
FPS = 30

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

TILE_COLORS = {
    x: ((255 - 15*(log(x)/log(2))),255,(255 - 15*(log(x)/log(2)))) for x in [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
}
TILE_COLORS[0] = GRAY  # Gray for empty tiles


In [ ]:
class Game2048:
    def __init__(self, size=GRID_SIZE):
        self.size = size
        self.board = np.zeros((self.size, self.size), dtype=int)
        self.moves = 0
        self.add_new_tile()
        self.add_new_tile()
        self.game_over = False
        self.has_won = False

        self.history = History(self)

    def add_new_tile(self):
        empty_cells = np.argwhere(self.board == 0)
        if empty_cells.size > 0:
            i, j = random.choice(empty_cells)
            self.board[i, j] = 2 if random.random() < 0.9 else 4

    def transpose(self):
        self.board = np.transpose(self.board)

    def reverse(self):
        for i in range(self.size):
            self.board[i] = np.flip(self.board[i])

    def slide(self, row):
        row = row[row != 0]
        return np.pad(row, (0, self.size - len(row)), 'constant')

    def merge(self, row):
        row = row.copy()
        i = 0
        while i < self.size - 1:
            if row[i] == row[i + 1] and row[i] != 0:
                row[i] *= 2
                row[i + 1] = 0
                i += 1
            i += 1
        return row

    def move_left(self):
        moved = False
        old_board = np.copy(self.board)
        for i in range(self.size):
            original_row = self.board[i].copy()
            row = self.slide(original_row)
            row = self.merge(row)
            row = self.slide(row)
            if not np.array_equal(row, original_row):
                moved = True
            self.board[i] = row
        if not moved:
            self.board = old_board
        return moved

    def move_right(self):
        old_board = np.copy(self.board)
        self.reverse()
        moved = self.move_left()
        self.reverse()
        if not moved:
            self.board = old_board
        return moved

    def move_up(self):
        old_board = np.copy(self.board)
        self.transpose()
        moved = self.move_left()
        self.transpose()
        if not moved:
            self.board = old_board
        return moved
    
    def move_down(self):
        old_board = np.copy(self.board)
        self.transpose()
        moved = self.move_right()
        self.transpose()
        if not moved:
            self.board = old_board
        return moved

    def is_game_over(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    return False
                if i < self.size - 1 and self.board[i][j] == self.board[i + 1][j]:
                    return False
                if j < self.size - 1 and self.board[i][j] == self.board[i][j + 1]:
                    return False
        return True

    def check_win(self):
        return np.any(self.board == 2048)

    def input(self, key):
        old_board = self.board.copy()

        if key == pygame.K_UP:
            self.history.save_state() #save the state before moving
            moved = self.move_up()

        elif key == pygame.K_DOWN:
            self.history.save_state() 
            moved = self.move_down()

        elif key == pygame.K_LEFT:
            self.history.save_state()
            moved = self.move_left()

        elif key == pygame.K_RIGHT:
            self.history.save_state()
            moved = self.move_right()
        else:
            moved = False

        if not np.array_equal(old_board, self.board):
            self.moves += 1
            self.add_new_tile()
            if self.is_game_over():
                self.game_over = True
            if self.check_win():
                self.has_won = True


In [ ]:
class History:
    def __init__(self, game):
        self.game = game
        self.states = []

    def save_state(self):
        # Save a deep copy of the current board state
        self.states.append((np.copy(self.game.board), self.game.moves))
        # Limit the history to the last 5 states
        if len(self.states) > 5:
            self.states.pop(0)
            print("popped") #Debugging

    def undo(self):
        if self.states:
            self.game.board, self.game.moves = self.states.pop()
            self.game.game_over = False
            self.game.has_won = False
        else:
            print("No more moves to undo.")

In [ ]:
# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2048")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 40)
large_font = pygame.font.Font(None, 80)

game = Game2048()
running = True 

In [ ]:
while running:
    clock.tick(FPS)
    #user input interaction
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if game.has_won:
                running = False  
            elif event.key == pygame.K_ESCAPE:
                running = False 
            elif event.key == pygame.K_z:
                game.history.undo()
            else:
                game.input(event.key)

    screen.fill(WHITE)

    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            value = game.board[i][j]
            color = TILE_COLORS.get(value, GRAY)
            rect = pygame.Rect(j * (TILE_SIZE + MARGIN) + MARGIN,
                               i * (TILE_SIZE + MARGIN) + MARGIN,
                               TILE_SIZE, TILE_SIZE)
            pygame.draw.rect(screen, color, rect)
            if value != 0:
                text = font.render(str(value), True, BLACK)
                text_rect = text.get_rect(center=rect.center)
                screen.blit(text, text_rect)

    moves_text = font.render(f"Moves: {game.moves}", True, BLACK)
    moves_rect = moves_text.get_rect(center=(WIDTH // 2, HEIGHT - 25))
    screen.blit(moves_text, moves_rect)

    if game.game_over:
        game_over_text = font.render("Game Over!", True, BLACK)
        game_over_rect = game_over_text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(game_over_text, game_over_rect)

    if game.has_won:
        win_text = large_font.render("You Win!", True, BLACK)
        win_rect = win_text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(win_text, win_rect)

    pygame.display.flip()

pygame.quit()
